In [1]:
using EDM4hep
using EDM4hep.RootIO
using StaticArrays
using LorentzVectorHEP
using JSON
using ONNXRunTime
using StructArrays

In [2]:
using Pkg
Pkg.resolve()
Pkg.develop(Pkg.PackageSpec(path = "/Users/harrywanghc/Developer/2025/JuliaHEPForkToMain/JetReconstruction.jl"))
using JetReconstruction
Pkg.status()

  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [44e8cb2c] ~ JetReconstruction v0.4.7 `~/Developer/2025/HarryFork/JetReconstruction.jl` ⇒ v0.5.0-dev `~/Developer/2025/JuliaHEPForkToMain/JetReconstruction.jl`
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [44e8cb2c] ~ JetReconstruction v0.4.7 `~/Developer/2025/HarryFork/JetReconstruction.jl` ⇒ v0.5.0-dev `~/Developer/2025/JuliaHEPForkToMain/JetReconstruction.jl`
ERROR: LoadError: ArgumentError: Package JetFlavourTagging does not have JSON in its dependencies:
- You may have a partially installed environment. Try `Pkg.instantiate()`
  to ensure all packages in the environment are installed.
- Or, if you have JetFlavourTagging checked out for development and have
  added JSON as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.reso

Status `~/.julia/environments/v1.11/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [a93c6f00] DataFrames v1.7.0
  [eb32b910] EDM4hep v0.4.3
⌃ [5c1252a2] GeometryBasics v0.5.7
  [7073ff75] IJulia v1.27.0
  [682c06a0] JSON v0.21.4
  [44e8cb2c] JetReconstruction v0.5.0-dev `~/Developer/2025/JuliaHEPForkToMain/JetReconstruction.jl`
  [98e50ef6] JuliaFormatter v2.1.2
⌃ [f612022c] LorentzVectorHEP v0.1.6
  [e034b28e] ONNXRunTime v1.3.1
  [91a5bcdd] Plots v1.40.13
  [90137ffa] StaticArrays v1.9.13
  [10745b16] Statistics v1.11.1
  [2913bbd2] StatsBase v0.34.5
⌅ [09ab397b] StructArrays v0.6.21
⌃ [1e6cf692] TestEnv v1.102.0
  [3cd96dde] UnROOT v0.10.36
  [37e2e46d] LinearAlgebra v1.11.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [3]:
# Paths to model files
model_dir = "data/wc_pt_7classes_12_04_2023"
onnx_path = joinpath(model_dir, "fccee_flavtagging_edm4hep_wc_v1.onnx")
json_path = joinpath(model_dir, "fccee_flavtagging_edm4hep_wc_v1.json")

# Load the configuration and model
config = JSON.parsefile(json_path)
model = ONNXRunTime.load_inference(onnx_path)

# Display the output classes we'll predict
println("The model predicts these flavor classes:")
for class_name in config["output_names"]
    println(" - ", class_name)
end

The model predicts these flavor classes:
 - recojet_isG
 - recojet_isQ
 - recojet_isS
 - recojet_isC
 - recojet_isB


In [4]:
# Path to ROOT file with EDM4hep data
edm4hep_path = "data/events_080263084.root"
reader = RootIO.Reader(edm4hep_path)

# Get event information
events = RootIO.get(reader, "events")
println("Loaded $(length(events)) events")

# Choose a specific event to analyze (event #15)
event_id = 15
evt = events[event_id]
println("Processing event #$event_id")

# Get reconstructed particles and tracks
recps = RootIO.get(reader, evt, "ReconstructedParticles")
tracks = RootIO.get(reader, evt, "EFlowTrack_1")

# Get needed collections for feature extraction
bz = RootIO.get(reader, evt, "magFieldBz", register = false)[1]
trackdata = RootIO.get(reader, evt, "EFlowTrack")
trackerhits = RootIO.get(reader, evt, "TrackerHits")
gammadata = RootIO.get(reader, evt, "EFlowPhoton")
nhdata = RootIO.get(reader, evt, "EFlowNeutralHadron")
calohits = RootIO.get(reader, evt, "CalorimeterHits")
dNdx = RootIO.get(reader, evt, "EFlowTrack_2")
track_L = RootIO.get(reader, evt, "EFlowTrack_L", register = false)

println("Loaded $(length(recps)) reconstructed particles")
println("Loaded $(length(tracks)) tracks")

Loaded 100000 events
Processing event #15
Loaded 65 reconstructed particles
Loaded 29 tracks


In [5]:
# Cluster jets using the EEkt algorithm with R=2.0 and p=1.0
cs = jet_reconstruct(recps; p = 1.0, R = 2.0, algorithm = JetAlgorithm.EEKt)

# Get 2 exclusive jets
jets = exclusive_jets(cs; njets=2, T=EEJet)

# For each jet, get its constituent particles
constituent_indices = [constituent_indexes(jet, cs) for jet in jets]
jet_constituents = build_constituents_cluster(recps, constituent_indices)

MethodError: MethodError: no method matching build_constituents_cluster(::StructVector{ReconstructedParticle, @NamedTuple{index::StructVector{ObjectID{ReconstructedParticle}, @NamedTuple{index::Vector{Int64}, collectionID::Vector{UInt32}}, Int64}, type::SubArray{Int32, 1, Vector{Int32}, Tuple{UnitRange{Int64}}, true}, energy::SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}, momentum::StructVector{Vector3f, @NamedTuple{x::SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}, y::SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}, z::SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}}, Int64}, referencePoint::StructVector{Vector3f, @NamedTuple{x::SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}, y::SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}, z::SubArray{Float32, 1, Vector{Float32}, Tuple{UnitRange{Int64}}, true}}, Int64}, charge::SubArray{Float32, 1, Vector{Float32}, Tupl

In [7]:
typeof(constituent_indices)

Vector{Vector{Int64}} (alias for Array{Array{Int64, 1}, 1})

In [ ]:
println("Extracting features for flavor tagging...")
feature_data = extract_features(
    jets, 
    jet_constituents, 
    tracks, 
    bz, 
    track_L, 
    trackdata, 
    trackerhits, 
    gammadata, 
    nhdata, 
    calohits, 
    dNdx
)
println("Step 1: Feature extraction completed.")

model, config = setup_weaver(
    onnx_path,
    json_path
)

println("Step 2: Weaver setup completed.")

input_tensors = prepare_input_tensor(
    jet_constituents,
    jets,
    config,
    feature_data
)

println("Step 3: Input tensor preparation completed.")

println("Running flavor tagging inference...")
weights = get_weights(
    0,  # Thread slot
    feature_data,
    jets,
    jet_constituents,
    config,
    model
)
println("Step 4: Weights retrieval completed.")

jet_scores = Dict{String, Vector{Float32}}()
for (i, score_name) in enumerate(config["output_names"])
    jet_scores[score_name] = get_weight(weights, i-1)
end

println("Jet scores:")
for (name, scores) in jet_scores
    println(" - $name: $(scores[1])")
end